# Example notebook to use the ee-water-extraction tool

First, you need to create a Google Cloud Project with the EarthEngine API enabled. 
Then create a service account with the EarthEngine Viewer role, and download a login json key for this service account. 

## Login to Google Cloud

The next cell load Earth Engine package and login to the API using the service account.

In [ ]:
import ee

credentials = ee.ServiceAccountCredentials('ee-water-extraction@earthengine-371715.iam.gserviceaccount.com', 
                                           './earthengine-key.json')
ee.Initialize(credentials)

## Classical usage workflow
### Import the required packages

In [ ]:
import geemap

from eewaterextraction import data_management
from eewaterextraction import classification
from eewaterextraction import visualization
from eewaterextraction import dgo_metrics

# The next lines are useful if you make modifications in the module files and want to reload it without restarting the notebook kernel
import importlib
importlib.reload(data_management)
importlib.reload(classification)
importlib.reload(visualization)
importlib.reload(dgo_metrics)

### Upload the DGOs to GEE

In [ ]:
# Set the local DGO file path
dgo_shapefile_path = './example_data/Lhasa_RC_DGO2km_updated.shp'

# Upload it to GEE
dgo_shp = geemap.shp_to_ee(dgo_shapefile_path)

# Create a region of interset (union of all the DGOs)
roi = dgo_shp.union(1)

### Classify images

In [ ]:
# Get the landsat image collection for your ROI
collection = data_management.getLandsatCollection(start=ee.Date('1980-01-01'), 
                                                  end=ee.Date('2100-01-01'), 
                                                  cloud_filter=80, # Maximum cloud coverage accepted (%)
                                                  cloud_masking=True, # Set to False if you don't want to mask the clouds on accepted images
                                                  roi=roi)

# Calculate MNDWI, NDVI and NDWI
collection = classification.calculateIndicators(collection)

# Classify the objects using the indicators
collection = classification.classifyObjects(collection)

### Layers visualisation

At this point of the workflow, you can create an interactive map for an individual landsat image to check all the previously calculated layers.
This feature is only available in a jupyter notebook.

In [ ]:
from ipywidgets import interact, fixed

# Get the landsat_product_id of the collection
landsat_ids = collection.aggregate_array('LANDSAT_PRODUCT_ID').getInfo()

map = interact(visualization.imageVisualization, 
         collection=fixed(collection), 
         dgo_shp=fixed(dgo_shp), 
         landsat_id=landsat_ids)

# Show the map in the notebook output
map

### Download layers

If your ROI is not too big, it's possible to download the layers to your local disk. See below the content of each output band.

| band number | content |   
|---|---|
| 1 | blue |
| 2 | green |
| 3 | red |
| 4 | nir |
| 5 | swir1 |
| 6 | swir2 |
| 7 | qa_pixel |
| 8 | mndwi |
| 9 | ndvi |
| 10 | ndwi |
| 11 | water |
| 12 | vegetation |
| 13 | active channel |


In [ ]:
data_management.imageDownload(collection=collection, 
                              landsat_id=landsat_id, 
                              roi=roi, 
                              scale=90, # Downgrading is recommended to reduce the file size
                              output='./example_data/landsat_export.tif')

### Calculate metrics
Execute one of the cell below to select DGOs

In [ ]:
# Select some DGOs
dgo_list = [57,35,42]
selected_dgo = dgo_shp.filter(ee.Filter.inList('DGO_FID', dgo_list))

In [ ]:
# Or select all the DGOs
selected_dgo = dgo_shp

Then calculate the metrics for those DGOs

In [ ]:
# Filter dates before metrics calculation
start = '2020-01-01'
end = '2020-03-31'

subcollection = collection.filterDate(ee.Date(start), ee.Date(end))

# Metrics calculation
metrics = dgo_metrics.calculateDGOsMetrics(collection=subcollection,
                                           dgos=selected_dgo)

#### Option 1

You can download calculated metrics as pandas dataframe. Technically, the metric calculation is made at this point of the workflow, so the computation time can be long.

In [ ]:
from itertools import chain
import pandas as pd

# Get metrics as pandas dataframe
local_data = metrics.getInfo()
metrics_df = pd.DataFrame(list(chain(*local_data)))

# Save a csv file
metrics_df.to_csv('./example_data/properties.csv')

#### Option 2

If you have too much DGOs and/or dates, you may need to do a month-to-month loop to reduce Google Cloud's request size. In this case, you can use the code below instead of the previous cell.

In [ ]:
import os
from itertools import chain
import pandas as pd

output_path = './example_data/'
output_prefix = 'properties_'

start[:3]
for year in range(int(start[:4]), int(end[:4])):
    for month in range(1, 13):
        collec_start = ee.Date(f'{year}-{month:0>2}-01')
        
        if month in [1,3,5,7,8,10,12]:
            collec_end = ee.Date(f'{year}-{month:0>2}-31')
        elif month in [4,6,9,11]:
            collec_end = ee.Date(f'{year}-{month:0>2}-30')
        elif month == 2:
            collec_end = ee.Date(f'{year}-03-01')
            
        # Filter dates before metrics calculation
        subcollection = collection.filterDate(collec_start, collec_end)

        metrics = dgo_metrics.calculateDGOsMetrics(collection=subcollection,
                                                   dgos=selected_dgo)
        
        # Get metrics as pandas dataframe
        local_data = metrics.getInfo()
        metrics_df = pd.DataFrame(list(chain(*local_data)))

        # Save a csv file
        output = os.path.join(output_path, f'{output_prefix}{year}-{month:0>2}.csv')
        metrics_df.to_csv(output)

## Metrics

| metric name | description |   
|---|---|
| AC_AREA | Active Channel area |
| CLOUD_SCORE | Percent of the DGO covered by clouds |
| COVERAGE_SCORE | Percent of the DGO covered by the Landsat image |
| MEAN_AC_MNDWI | Mean MNDWI in the active channel surface |
| MEAN_AC_NDVI | Mean NDVI in the active channel surface |
| MEAN_DRY_MNDWI | Mean MNDWI in the surface which is not water  |
| MEAN_MNDWI | Mean MNDWI of the full DGO |
| MEAN_NDVI| Mean NDVI of the full DGO |
| MEAN_VEGETATION_MNDWI | Mean MNDWI in the vegetation surface |
| MEAN_VEGETATION_NDVI | Mean NDVI in the vegetation surface |
| MEAN_WATER_MNDWI | Mean MNDWI in the water surface |
| VEGETATION_AREA | Vegetation area |
| VEGETATION_PERIMETER | Vegetation surface perimeter |
| WATER_AREA | Water area |
| WATER_PERIMETER | Water surface perimeter |